# Data Loading

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
pd.set_option('display.float_format','{:.2f}'.format)

In [3]:
nonesales = pd.read_parquet('../Data/nonesales_pivoted_2025_cleaned_v2.parquet')

In [4]:
esales = pd.read_parquet('../Data/esales_pivoted_2025_cleaned_v2.parquet')

In [5]:
def get_month_list(tp):
    yy, mon_str, k_str = tp.split('_')
    k = int(k_str)
    base_year = 2000 + int(yy)
    start_date = datetime.strptime(f'{mon_str}_{base_year - 1}', '%b_%Y') + relativedelta(months=1)
    return [(start_date + relativedelta(months=i)).strftime('%Y-%m') for i in range(k)]

# Time Level 12 months apr-24 to mar-25

## Time Level Definition

In [6]:
tp1 = '24_jun_12'
tp2 = '25_jun_12'


tp1_cols = get_month_list(tp1)
tp2_cols = get_month_list(tp2)

In [7]:
esales_df = esales[['global_id','flag_gl_id'] + tp1_cols + tp2_cols]
nonesales_df = nonesales[['global_id','flag_gl_id'] + tp1_cols + tp2_cols]

## Pre Analysis

### Analysis Pre steps 

In [8]:
def calc_metrics(df, tp1, tp2, tp1_cols, tp2_cols):
    df = df.copy()
    tp1_array = df[tp1_cols].to_numpy()
    tp2_array = df[tp2_cols].to_numpy()

    tp1_total = tp1_array.sum(axis=1)
    tp2_total = tp2_array.sum(axis=1)

    tp1_active = (tp1_array != 0).sum(axis=1)
    tp2_active = (tp2_array != 0).sum(axis=1)

    tp1_avg = np.divide(tp1_total, tp1_active, out=np.zeros_like(tp1_total, dtype=float), where=tp1_active != 0)
    tp2_avg = np.divide(tp2_total, tp2_active, out=np.zeros_like(tp2_total, dtype=float), where=tp2_active != 0)

    def masked_std(arr):
        masked = np.ma.masked_where(arr == 0, arr)
        return masked.std(axis=1, ddof=0)

    tp1_std = masked_std(tp1_array)
    tp2_std = masked_std(tp2_array)

    df[f'{tp1}_Total'] = tp1_total
    df[f'{tp2}_Total'] = tp2_total
    df[f'months_active_{tp1}'] = tp1_active
    df[f'months_active_{tp2}'] = tp2_active
    df[f'avg_act_monthly_{tp1}'] = tp1_avg
    df[f'avg_act_monthly_{tp2}'] = tp2_avg
    df[f'std_dev_{tp1}'] = tp1_std
    df[f'std_dev_{tp2}'] = tp2_std

    return df

In [9]:
esales_df = calc_metrics(esales_df,tp1, tp2,tp1_cols,tp2_cols)
nonesales_df = calc_metrics(nonesales_df,tp1, tp2,tp1_cols,tp2_cols)

In [10]:
merged_df = pd.merge(nonesales_df, esales_df, on='global_id', how='outer', suffixes=('_nonesales', '_esales'))
merged_df['flag_gl_id'] = merged_df['flag_gl_id_esales'].combine_first(merged_df['flag_gl_id_nonesales'])
merged_df.drop(columns=['flag_gl_id_esales','flag_gl_id_nonesales'],inplace=True)
merged_df.fillna(0, inplace=True)

usag_df = merged_df

In [11]:
usag_df


,global_id,2023-07_nonesales,2023-08_nonesales,2023-09_nonesales,2023-10_nonesales,2023-11_nonesales,2023-12_nonesales,2024-01_nonesales,2024-02_nonesales,2024-03_nonesales,...,2025-06_esales,24_jun_12_Total_esales,25_jun_12_Total_esales,months_active_24_jun_12_esales,months_active_25_jun_12_esales,avg_act_monthly_24_jun_12_esales,avg_act_monthly_25_jun_12_esales,std_dev_24_jun_12_esales,std_dev_25_jun_12_esales,flag_gl_id
0,000006c3-c93c-b2d1-75a9-d55eff943df8,139.24,76.22,0.00,0.00,25.85,59.40,0.00,43.83,14.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,000008fd-b89a-e34f-4cc1-8082184f9627,63.72,126.16,97.98,14.09,411.39,140.32,48.02,150.56,68.61,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,00000d0f-4cff-2258-96e3-37fb60e6dd3f,181.28,87.38,131.72,66.94,89.62,248.63,316.88,531.97,621.99,...,104.00,0.00,226.26,0.00,2.00,0.00,113.13,0.00,9.13,0.00
3,00001b6a-9f19-d466-5b17-498b2134e54b,3294.76,1587.71,1871.09,140.99,-130.99,0.00,166.20,-124.99,0.00,...,0.00,1903.89,0.00,5.00,0.00,380.78,0.00,174.82,0.00,0.00
4,00003208-43d2-548e-6834-1d26ca82306f,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698982,ffffe0b8-4419-19cc-ebca-e9f175b9a7b3,0.00,0.00,0.00,85.47,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1698983,ffffebc6-2076-9b23-51c2-5f5293338412,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1698984,fffff3c4-c07a-8d6a-8d98-a74f28d06baf,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1698985,fffffc8f-cb07-b227-ef84-b2b43815c1ff,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Analysis

### Distribution analysis

In [12]:
def assign_segment(df, k, stype, atype, tp1, tp2):
    avg_tp1 = df[f'avg_{atype}_monthly_{tp1}_{stype}']
    avg_tp2 = df[f'avg_{atype}_monthly_{tp2}_{stype}']
    std_tp1 = df[f'std_dev_{tp1}_{stype}']
    months_tp1 = df[f'months_active_{tp1}_{stype}']

    conditions = [
        (avg_tp1 == 0) & (avg_tp2 == 0),
        avg_tp1 == 0,
        avg_tp2 == 0,
        months_tp1 <= 1,
        avg_tp2 < avg_tp1 - k * std_tp1,
        avg_tp2 > avg_tp1 + k * std_tp1
    ]

    choices = [
        "Not in Channel",
        "New",
        "Declining",
        "Unallocated",
        "Declining",
        "Growing"
    ]

    return np.select(conditions, choices, default="Stable")


In [13]:
stypes = ['esales', 'nonesales']
atypes = ['act']
ks = [1]

for stype in stypes:
    for atype in atypes:
        for k in ks:
            usag_df[f'{tp1}_{tp2}_{atype}_mon_{stype}_sd_{str(k)}_segment'] = assign_segment(usag_df,k, stype, atype, tp1, tp2)
            print(f'{tp1}_{tp2}_{atype}_mon_{stype}_sd_{str(k)}_segment done')


24_jun_12_25_jun_12_act_mon_esales_sd_1_segment done
24_jun_12_25_jun_12_act_mon_nonesales_sd_1_segment done


In [14]:
usag_segments = usag_df[['global_id',f'{tp1}_{tp2}_act_mon_esales_sd_1_segment',f'{tp1}_{tp2}_act_mon_nonesales_sd_1_segment']]

### YoY assign

In [15]:
def compute_yoy(df, stype, tp1, tp2):
    total_tp1 = df[f'{tp1}_Total_{stype}']
    total_tp2 = df[f'{tp2}_Total_{stype}']
    
    cond1 = (total_tp1 <= 2) & (total_tp2 <= 2)
    cond2 = (total_tp1 <= 2) & (total_tp2 > 2)
    
    conditions = [cond1, cond2]
    choices = [0, 1]

    yoy = np.select(conditions, choices, default=(total_tp2 / total_tp1) - 1)
    
    return yoy

In [16]:
usag_df[f'{tp1}_{tp2}_esales_yoy'] = compute_yoy(usag_df,'esales', tp1, tp2)
usag_df[f'{tp1}_{tp2}_nonesales_yoy'] = compute_yoy(usag_df,'nonesales', tp1, tp2)

## New Data Load

In [17]:
bucket_bins = [-np.inf,0,0.001,500,1000,2500, 5000, 10000, 25000, np.inf]
buckets = ['<0','0','0-500','500-1000','1000-2500','2500-5000', '5000-10000', '10000-25000', '25000+']

usag_df[f'{tp1}_bucket'] = pd.cut(usag_df[f'{tp1}_Total_nonesales'], bins=bucket_bins, labels=buckets, right=False)

## Inc Analysis

In [18]:
stypes = ['esales', 'nonesales']
atypes = ['act']
ks = [1]

In [19]:
usag_df[f'{tp1}_bucket'] = usag_df[f'{tp1}_bucket'].astype(str)

In [20]:
nonesales_buckets = usag_df.groupby([f'{tp1}_bucket'])[[f'{tp1}_Total_nonesales',f'{tp2}_Total_nonesales']].sum().reset_index()
nonesales_buckets['comp_bucket_YoY'] = (nonesales_buckets[f'{tp2}_Total_nonesales']/nonesales_buckets[f'{tp1}_Total_nonesales'])-1
nonesales_buckets = nonesales_buckets[[f'{tp1}_bucket','comp_bucket_YoY']]
usag_df = usag_df.merge(nonesales_buckets,how='left',on=[f'{tp1}_bucket'])

In [21]:
cust_buckets = usag_df.groupby([f'{tp1}_bucket','flag_gl_id'])['global_id'].count().reset_index()

In [22]:
rev_buckets = usag_df.groupby([f'{tp1}_bucket','flag_gl_id'])[[f'{tp1}_Total_esales',f'{tp2}_Total_esales',f'{tp1}_Total_nonesales',f'{tp2}_Total_nonesales']].sum().reset_index()

In [23]:
rev_buckets = rev_buckets.merge(cust_buckets, on=[f'{tp1}_bucket','flag_gl_id'],how='inner')
rev_buckets['esales_yoy'] = (rev_buckets[f'{tp2}_Total_esales']/rev_buckets[f'{tp1}_Total_esales'])-1
rev_buckets['nonesales_yoy'] = (rev_buckets[f'{tp2}_Total_nonesales']/rev_buckets[f'{tp1}_Total_nonesales'])-1

In [24]:
rev_buckets

,24_jun_12_bucket,flag_gl_id,24_jun_12_Total_esales,25_jun_12_Total_esales,24_jun_12_Total_nonesales,25_jun_12_Total_nonesales,global_id,esales_yoy,nonesales_yoy
0,0,0.00,2119108.28,73149212.81,0.00,328727902.02,354398,33.52,493486285681185587200.00
1,0,1.00,199114.22,17237051.06,0.00,59206922.91,97006,85.57,219279832200024424448.00
2,0-500,0.00,39453869.11,52649045.06,79538018.14,147687647.62,443116,0.33,0.86
3,0-500,1.00,3310048.28,3792239.78,13129976.54,9398091.04,83859,0.15,-0.28
4,1000-2500,0.00,128958136.25,133377849.55,306709314.72,322143655.04,189407,0.03,0.05
5,1000-2500,1.00,9010085.70,8316086.80,31511776.62,14478292.36,19715,-0.08,-0.54
6,10000-25000,0.00,528188893.71,519242407.08,1048002589.36,954919343.89,67457,-0.02,-0.09
7,10000-25000,1.00,25932145.08,22633418.36,77544829.34,44936057.43,5044,-0.13,-0.42
8,2500-5000,0.00,189628545.07,186759425.49,412056965.34,400254985.45,115949,-0.02,-0.03
9,2500-5000,1.00,11946134.40,11175507.24,37357376.91,16406037.98,10564,-0.06,-0.56


In [25]:
rev_buckets.to_excel('revenue buckets summary.xlsx')

In [26]:
# esales_buckets.to_excel('esales_buckets.xlsx')
nonesales_buckets

,24_jun_12_bucket,comp_bucket_YoY
0,0,414398276327985446912.00
1,0-500,0.70
2,1000-2500,-0.00
3,10000-25000,-0.11
4,2500-5000,-0.07
5,25000+,-0.09
6,500-1000,0.13
7,5000-10000,-0.09
8,<0,-0.87


In [27]:
nonesales_buckets.to_excel('nonesales_buckets_segment.xlsx')

In [28]:
def compute_increment(df, seg_col, tp1, tp2):
    yoy_col = f'{tp1}_{tp2}_nonesales_yoy'
    bucket_col = f'{tp1}_bucket'
    comp_yoy_col = 'comp_bucket_YoY'

    conditions = [
        df[seg_col] == 'Not in Channel',
        df[bucket_col].isin(['<0', '0'])
    ]
    
    choices = [
        0.0,
        df[yoy_col]
    ]
    
    inc_pct = np.select(conditions, choices, default=df[yoy_col] - df[comp_yoy_col])
    return inc_pct

In [29]:
seg_col = f'{tp1}_{tp2}_act_mon_nonesales_sd_1_segment'
cust_count = usag_df[(usag_df[seg_col]=='Not in Channel')].shape[0]
cust_count = len(usag_df) - cust_count
cust_count

1517566

In [30]:
usag_df[f'{tp1}_{tp2}_nonesales_incremental_pct'] = compute_increment(usag_df,seg_col, tp1, tp2)
usag_df[f'{tp1}_{tp2}_nonesales_incremental_abs'] = usag_df[f'{tp1}_{tp2}_nonesales_incremental_pct'] * usag_df[f'{tp1}_Total_nonesales']
print(usag_df[f'{tp1}_{tp2}_nonesales_incremental_abs'].sum())

-8973866.859999973


### Placeholder

In [31]:
# usag_df['STO__OWNERSHIP_TYPE_CODE'] = usag_df['STO__OWNERSHIP_TYPE_CODE'].astype(str)

In [32]:
usag_df.to_parquet(f'Outputs/usag_{tp1}_{tp2}_bucket_isocos_pneinc_updated.parquet',index=False)

## Post Analysis

### Crosstab creation

In [33]:

segments = ['Unallocated', 'New', 'Growing', 'Stable', 'Declining', 'Not in Channel']
segments

['Unallocated', 'New', 'Growing', 'Stable', 'Declining', 'Not in Channel']

In [34]:
def crosstab_func(usag_df,atype,k,tp1,tp2):
    metric_rows = ['# of Customers', 'noneSales_Inc', f'eSales {tp1}', f'eSales {tp2}']
    metric_cols = ['.','..', f'non-eSales {tp1}', f'non-eSales {tp2}']


    row_index = pd.MultiIndex.from_product([segments, metric_rows])
    col_index = pd.MultiIndex.from_product([segments, metric_cols])


    cross_tab = pd.DataFrame(index=row_index, columns=col_index)


    total_nonesales_tp1 = usag_df[f'{tp1}_Total_nonesales'].sum()
    total_nonesales_tp2 = usag_df[f'{tp2}_Total_nonesales'].sum()
    total_esales_tp1 = usag_df[f'{tp1}_Total_esales'].sum()
    total_esales_tp2 = usag_df[f'{tp2}_Total_esales'].sum()

    for e_seg in segments:
        for n_seg in segments:
            subset = usag_df[
                (usag_df[f'{tp1}_{tp2}_{atype}_mon_esales_sd_{str(k)}_segment'] == e_seg) &
                (usag_df[f'{tp1}_{tp2}_{atype}_mon_nonesales_sd_{str(k)}_segment'] == n_seg)
            ]

            num_customers = subset.shape[0]
            esales_tp1 = subset[f'{tp1}_Total_esales'].sum()
            esales_tp2 = subset[f'{tp2}_Total_esales'].sum()
            nonesales_tp1 = subset[f'{tp1}_Total_nonesales'].sum()
            nonesales_tp2 = subset[f'{tp2}_Total_nonesales'].sum()

            nonesales_inc = subset[f'{tp1}_{tp2}_nonesales_incremental_abs'].sum()

            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, '.')] = num_customers
            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, f'non-eSales {tp1}')] = f"{nonesales_tp1:.0f}"
            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, f'non-eSales {tp2}')] = f"{nonesales_tp2:.0f}"
            cross_tab.loc[(e_seg, 'noneSales_Inc'), (n_seg, f'non-eSales {tp1}')] = f"{(nonesales_tp1/total_nonesales_tp1):.2%}"
            cross_tab.loc[(e_seg, 'noneSales_Inc'), (n_seg, f'non-eSales {tp2}')] = f"{(nonesales_tp2/total_nonesales_tp2):.2%}"

            cross_tab.loc[(e_seg, 'noneSales_Inc'), (n_seg, '.')] = f"{nonesales_inc:.0f}"

            cross_tab.loc[(e_seg, f'eSales {tp1}'), (n_seg, '.')] = f"{esales_tp1:.0f}"
            cross_tab.loc[(e_seg, f'eSales {tp2}'), (n_seg, '.')] = f"{esales_tp2:.0f}"
            cross_tab.loc[(e_seg, f'eSales {tp1}'), (n_seg, '..')] = f"{(esales_tp1/total_esales_tp1):.2%}"
            cross_tab.loc[(e_seg, f'eSales {tp2}'), (n_seg, '..')] = f"{(esales_tp2/total_esales_tp2):.2%}"


    return cross_tab

In [35]:
sheet_dict = {}
for atype in atypes:
    for k in ks:
        sheet_name = f"cross_tab_{tp1}_{tp2}_{atype}_mon_sd_{str(k)}"
        sheet_dict[sheet_name] = crosstab_func(usag_df,atype, k,tp1, tp2)
print("done")

done


In [36]:
sheet_dict[f'cross_tab_{tp1}_{tp2}_act_mon_sd_1']

Unallocated                              \
                                          .     .. non-eSales 24_jun_12   
Unallocated    # of Customers           891    NaN               435780   
               noneSales_Inc        3398888    NaN                0.01%   
               eSales 24_jun_12      443063  0.02%                  NaN   
               eSales 25_jun_12     6854879  0.33%                  NaN   
New            # of Customers          1518    NaN               825742   
               noneSales_Inc        3877361    NaN                0.01%   
               eSales 24_jun_12           0  0.00%                  NaN   
               eSales 25_jun_12     4051237  0.19%                  NaN   
Growing        # of Customers           484    NaN                99880   
               noneSales_Inc         724208    NaN                0.00%   
               eSales 24_jun_12      326707  0.02%                  NaN   
               eSales 25_jun_12     2651397  0.13%                  NaN   
Stable         # of Customers          1087    NaN               215817   
               noneSales_Inc        1047810    NaN                0.00%   
               eSales 24_jun_12     1784236  0.09%                  NaN   
               eSales 25_jun_12     2118194  0.10%                  NaN   
Declining      # of Customers           741    NaN               163090   
               noneSales_Inc         148555    NaN                0.00%   
               eSales 24_jun_12      468009  0.02%                  NaN   
               eSales 25_jun_12      155910  0.01%                  NaN   
Not in Channel # of Customers         94607    NaN             21163788   
               noneSales_Inc       34798122    NaN                0.33%   
               eSales 24_jun_12           0  0.00%                  NaN   
               eSales 25_jun_12           0  0.00%                  NaN   

                                                           New         \
                                non-eSales 25_jun_12         .     ..   
Unallocated    # of Customers                3923478       537    NaN   
               noneSales_Inc                   0.06%         0    NaN   
               eSales 24_jun_12                  NaN    102237  0.01%   
               eSales 25_jun_12                  NaN   1681324  0.08%   
New            # of Customers                4897230     15306    NaN   
               noneSales_Inc                   0.08%         0    NaN   
               eSales 24_jun_12                  NaN         0  0.00%   
               eSales 25_jun_12                  NaN  85136058  4.06%   
Growing        # of Customers                 897973       324    NaN   
               noneSales_Inc                   0.01%         0    NaN   
               eSales 24_jun_12                  NaN    168930  0.01%   
               eSales 25_jun_12                  NaN   1157644  0.06%   
Stable         # of Customers                1385284       628    NaN   
               noneSales_Inc                   0.02%         0    NaN   
               eSales 24_jun_12                  NaN    800552  0.04%   
               eSales 25_jun_12                  NaN    998678  0.05%   
Declining      # of Customers                 381772       505    NaN   
               noneSales_Inc                   0.01%         0    NaN   
               eSales 24_jun_12                  NaN    239903  0.01%   
               eSales 25_jun_12                  NaN     60216  0.00%   
Not in Channel # of Customers               62649653    252644    NaN   
               noneSales_Inc                   1.00%         0    NaN   
               eSales 24_jun_12                  NaN         0  0.00%   
               eSales 25_jun_12                  NaN         0  0.00%   

                                                                           \
                                non-eSales 24_jun_12 non-eSales 25_jun_12   
Unallocated    # of Customer

In [37]:
with pd.ExcelWriter(f"crosstab_output_bucket_pne_{tp1}_{tp2}.xlsx") as writer:
    for sheet_name, df in sheet_dict.items():
        df.to_excel(writer,sheet_name = sheet_name[:31])

### Placeholder

# Time Level 9 months apr-24 to dec-24

In [38]:
usag_df_proxy = usag_segments#pd.read_csv(f'usag_{tp1}_{tp2}_bucket_peinc_updated.csv')

In [39]:
tp_proxy_1 = tp1
tp_proxy_2 = tp2

In [40]:
# usag_df_proxy.to_csv(f'usag_{tp_proxy_1}_{tp_proxy_2}_peinc_updated.csv')

## Time Level Definition

In [41]:
tp1 = '24_jun_9'
tp2 = '25_jun_9'


tp1_cols = get_month_list(tp1)
tp2_cols = get_month_list(tp2)

In [42]:
esales_df = esales_df[['global_id','flag_gl_id'] + tp1_cols + tp2_cols]
nonesales_df = nonesales_df[['global_id','flag_gl_id'] + tp1_cols + tp2_cols]

## Pre Analysis

### Analysis Pre steps 

In [43]:
def calc_metrics(df, tp1, tp2, tp1_cols, tp2_cols):
    df = df.copy()
    tp1_array = df[tp1_cols].to_numpy()
    tp2_array = df[tp2_cols].to_numpy()

    tp1_total = tp1_array.sum(axis=1)
    tp2_total = tp2_array.sum(axis=1)

    tp1_active = (tp1_array != 0).sum(axis=1)
    tp2_active = (tp2_array != 0).sum(axis=1)

    tp1_avg = np.divide(tp1_total, tp1_active, out=np.zeros_like(tp1_total, dtype=float), where=tp1_active != 0)
    tp2_avg = np.divide(tp2_total, tp2_active, out=np.zeros_like(tp2_total, dtype=float), where=tp2_active != 0)

    def masked_std(arr):
        masked = np.ma.masked_where(arr == 0, arr)
        return masked.std(axis=1, ddof=0)

    tp1_std = masked_std(tp1_array)
    tp2_std = masked_std(tp2_array)

    df[f'{tp1}_Total'] = tp1_total
    df[f'{tp2}_Total'] = tp2_total
    df[f'months_active_{tp1}'] = tp1_active
    df[f'months_active_{tp2}'] = tp2_active
    df[f'avg_act_monthly_{tp1}'] = tp1_avg
    df[f'avg_act_monthly_{tp2}'] = tp2_avg
    df[f'std_dev_{tp1}'] = tp1_std
    df[f'std_dev_{tp2}'] = tp2_std

    return df

In [44]:
esales_df = calc_metrics(esales_df,tp1, tp2,tp1_cols,tp2_cols)
nonesales_df = calc_metrics(nonesales_df,tp1, tp2,tp1_cols,tp2_cols)

In [45]:
merged_df = pd.merge(nonesales_df, esales_df, on='global_id', how='outer', suffixes=('_nonesales', '_esales'))
merged_df['flag_gl_id'] = merged_df['flag_gl_id_esales'].combine_first(merged_df['flag_gl_id_nonesales'])
merged_df.drop(columns=['flag_gl_id_esales','flag_gl_id_nonesales'],inplace=True)
merged_df.fillna(0, inplace=True)

usag_df = merged_df

In [46]:
# usag_df.to_csv(f"usag_{tp1}_{tp2}_unflagged.csv")

In [47]:
# months_2022, months_2023,months_2024,months_2025 = get_year_columns(nonesales_df)
# months = months_2022 + months_2023 + months_2024 + months_2025



# for month in months:
#     usag_df[month] = usag_df[f'{month}_nonesales'] + usag_df[f'{month}_esales']

# # usag_df = usag_df[['global_id'] + months]


In [48]:
usag_df


,global_id,2023-07_nonesales,2023-08_nonesales,2023-09_nonesales,2023-10_nonesales,2023-11_nonesales,2023-12_nonesales,2024-01_nonesales,2024-02_nonesales,2024-03_nonesales,...,2025-03_esales,24_jun_9_Total_esales,25_jun_9_Total_esales,months_active_24_jun_9_esales,months_active_25_jun_9_esales,avg_act_monthly_24_jun_9_esales,avg_act_monthly_25_jun_9_esales,std_dev_24_jun_9_esales,std_dev_25_jun_9_esales,flag_gl_id
0,000006c3-c93c-b2d1-75a9-d55eff943df8,139.24,76.22,0.00,0.00,25.85,59.40,0.00,43.83,14.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,000008fd-b89a-e34f-4cc1-8082184f9627,63.72,126.16,97.98,14.09,411.39,140.32,48.02,150.56,68.61,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,00000d0f-4cff-2258-96e3-37fb60e6dd3f,181.28,87.38,131.72,66.94,89.62,248.63,316.88,531.97,621.99,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00001b6a-9f19-d466-5b17-498b2134e54b,3294.76,1587.71,1871.09,140.99,-130.99,0.00,166.20,-124.99,0.00,...,0.00,1215.23,0.00,3.00,0.00,405.08,0.00,204.64,0.00,0.00
4,00003208-43d2-548e-6834-1d26ca82306f,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698982,ffffe0b8-4419-19cc-ebca-e9f175b9a7b3,0.00,0.00,0.00,85.47,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1698983,ffffebc6-2076-9b23-51c2-5f5293338412,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1698984,fffff3c4-c07a-8d6a-8d98-a74f28d06baf,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1698985,fffffc8f-cb07-b227-ef84-b2b43815c1ff,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Analysis

### Distribution analysis

In [49]:
def get_segment(row,k,stype,atype,tp1,tp2):
    # print(f'avg_{atype}_monthly_2022_{stype}')
    if row[f'avg_{atype}_monthly_{tp1}_{stype}'] == 0 and row[f'avg_{atype}_monthly_{tp2}_{stype}'] == 0:
        return "Not in Channel"
    elif row[f'avg_{atype}_monthly_{tp1}_{stype}'] == 0:
        return "New"
    elif row[f'avg_{atype}_monthly_{tp2}_{stype}'] == 0:
        return "Declining"
    elif row[f'months_active_{tp1}_{stype}'] <= 1:
        return "Unallocated"
    elif row[f'avg_{atype}_monthly_{tp2}_{stype}'] < row[f'avg_{atype}_monthly_{tp1}_{stype}'] - k * row[f'std_dev_{tp1}_{stype}']:
        return "Declining"
    elif row[f'avg_{atype}_monthly_{tp2}_{stype}'] > row[f'avg_{atype}_monthly_{tp1}_{stype}'] + k * row[f'std_dev_{tp1}_{stype}']:
        return "Growing"
    else:
        return "Stable"


In [50]:
stypes = ['esales', 'nonesales']
atypes = ['act']
ks = [1]

# for stype in stypes:
#     for atype in atypes:
#         for k in ks:
#             usag_df[f'{tp1}_{tp2}_{atype}_mon_{stype}_sd_{str(k)}_segment'] = usag_df.apply(lambda row: get_segment(row,k, stype,atype,tp1,tp2), axis=1 )
#             print(f'{tp1}_{tp2}_{atype}_mon_{stype}_sd_{str(k)}_segment done')


### YoY assign

In [51]:
def compute_yoy(df, stype, tp1, tp2):
    total_tp1 = df[f'{tp1}_Total_{stype}']
    total_tp2 = df[f'{tp2}_Total_{stype}']
    
    cond1 = (total_tp1 <= 2) & (total_tp2 <= 2)
    cond2 = (total_tp1 <= 2) & (total_tp2 > 2)
    
    conditions = [cond1, cond2]
    choices = [0, 1]

    yoy = np.select(conditions, choices, default=(total_tp2 / total_tp1) - 1)
    
    return yoy

In [52]:
usag_df[f'{tp1}_{tp2}_esales_yoy'] = compute_yoy(usag_df,'esales', tp1, tp2)
usag_df[f'{tp1}_{tp2}_nonesales_yoy'] = compute_yoy(usag_df,'nonesales', tp1, tp2)

In [53]:
# usag_df.to_csv(f"usag_{tp1}_{tp2}_flagged.csv")

## Inc Analysis

In [54]:
# usag_df = pd.read_csv(f"usag_{tp1}_{tp2}_flagged.csv")

stypes = ['esales', 'nonesales']
atypes = ['act']
ks = [1]

In [55]:
usag_segments

,global_id,24_jun_12_25_jun_12_act_mon_esales_sd_1_segment,24_jun_12_25_jun_12_act_mon_nonesales_sd_1_segment
0,000006c3-c93c-b2d1-75a9-d55eff943df8,Not in Channel,Growing
1,000008fd-b89a-e34f-4cc1-8082184f9627,Not in Channel,Stable
2,00000d0f-4cff-2258-96e3-37fb60e6dd3f,New,Stable
3,00001b6a-9f19-d466-5b17-498b2134e54b,Declining,Stable
4,00003208-43d2-548e-6834-1d26ca82306f,Not in Channel,Not in Channel
...,...,...,...
1698982,ffffe0b8-4419-19cc-ebca-e9f175b9a7b3,Not in Channel,Declining
1698983,ffffebc6-2076-9b23-51c2-5f5293338412,Not in Channel,New
1698984,fffff3c4-c07a-8d6a-8d98-a74f28d06baf,Not in Channel,New
1698985,fffffc8f-cb07-b227-ef84-b2b43815c1ff,Not in Channel,New


In [56]:
usag_df_proxy_2 = usag_df_proxy[['global_id',f'{tp_proxy_1}_{tp_proxy_2}_act_mon_esales_sd_1_segment',f'{tp_proxy_1}_{tp_proxy_2}_act_mon_nonesales_sd_1_segment']]

In [57]:
usag_df=usag_df.merge(usag_df_proxy_2,how='inner',on='global_id')

In [58]:
bucket_bins = [-np.inf,0,0.001,500,1000,2500, 5000, 10000, 25000, np.inf]
buckets = ['<0','0','0-500','500-1000','1000-2500','2500-5000', '5000-10000', '10000-25000', '25000+']

usag_df[f'{tp1}_bucket'] = pd.cut(usag_df[f'{tp1}_Total_nonesales'], bins=bucket_bins, labels=buckets, right=False)

In [59]:
usag_df[f'{tp1}_bucket'] = usag_df[f'{tp1}_bucket'].astype(str)

In [60]:
nonesales_buckets = usag_df.groupby([f'{tp1}_bucket'])[[f'{tp1}_Total_nonesales',f'{tp2}_Total_nonesales']].sum().reset_index()
nonesales_buckets['comp_bucket_YoY'] = (nonesales_buckets[f'{tp2}_Total_nonesales']/nonesales_buckets[f'{tp1}_Total_nonesales'])-1
nonesales_buckets = nonesales_buckets[[f'{tp1}_bucket','comp_bucket_YoY']]
usag_df = usag_df.merge(nonesales_buckets,how='left',on=[f'{tp1}_bucket'])

In [61]:
# esales_buckets.to_excel('esales_buckets_9months.xlsx')
nonesales_buckets

,24_jun_9_bucket,comp_bucket_YoY
0,0,310039595412910571520.00
1,0-500,0.63
2,1000-2500,-0.05
3,10000-25000,-0.13
4,2500-5000,-0.09
5,25000+,-0.10
6,500-1000,0.06
7,5000-10000,-0.11
8,<0,-0.95


In [62]:
def compute_increment(df, seg_col, tp1, tp2):
    yoy_col = f'{tp1}_{tp2}_nonesales_yoy'
    bucket_col = f'{tp1}_bucket'
    comp_yoy_col = 'comp_bucket_YoY'

    conditions = [
        df[seg_col] == 'Not in Channel',
        df[bucket_col].isin(['<0', '0'])
    ]
    
    choices = [
        0.0,
        df[yoy_col]
    ]
    
    inc_pct = np.select(conditions, choices, default=df[yoy_col] - df[comp_yoy_col])
    return inc_pct

In [63]:
seg_col = f'{tp_proxy_1}_{tp_proxy_2}_act_mon_nonesales_sd_1_segment'
cust_count = usag_df[(usag_df[seg_col]=='Not in Channel')].shape[0]
cust_count = len(usag_df) - cust_count
cust_count

1517566

In [64]:
usag_df[f'{tp1}_{tp2}_nonesales_incremental_pct'] = compute_increment(usag_df,seg_col, tp1, tp2)
usag_df[f'{tp1}_{tp2}_nonesales_incremental_abs'] = usag_df[f'{tp1}_{tp2}_nonesales_incremental_pct'] * usag_df[f'{tp1}_Total_nonesales']
print(usag_df[f'{tp1}_{tp2}_nonesales_incremental_abs'].sum())

-6457498.819994665


### Placeholder

In [65]:
# usag_df['STO__OWNERSHIP_TYPE_CODE'] = usag_df['STO__OWNERSHIP_TYPE_CODE'].astype(str)

In [66]:
usag_df.to_parquet(f'Outputs/usag_{tp1}_{tp2}_bucket_isocos_pneinc_updated.parquet',index=False)

## Post Analysis

### Crosstab creation

In [67]:

segments = ['Unallocated', 'New', 'Growing', 'Stable', 'Declining', 'Not in Channel']
segments

['Unallocated', 'New', 'Growing', 'Stable', 'Declining', 'Not in Channel']

In [68]:
def crosstab_func(usag_df,atype,k,tp1,tp2,tp_proxy_1,tp_proxy_2):
    metric_rows = ['# of Customers', 'noneSales_Inc', f'eSales {tp1}', f'eSales {tp2}']
    metric_cols = ['.','..', f'non-eSales {tp1}', f'non-eSales {tp2}']


    row_index = pd.MultiIndex.from_product([segments, metric_rows])
    col_index = pd.MultiIndex.from_product([segments, metric_cols])


    cross_tab = pd.DataFrame(index=row_index, columns=col_index)


    total_nonesales_tp1 = usag_df[f'{tp1}_Total_nonesales'].sum()
    total_nonesales_tp2 = usag_df[f'{tp2}_Total_nonesales'].sum()
    total_esales_tp1 = usag_df[f'{tp1}_Total_esales'].sum()
    total_esales_tp2 = usag_df[f'{tp2}_Total_esales'].sum()

    for e_seg in segments:
        for n_seg in segments:
            subset = usag_df[
                (usag_df[f'{tp_proxy_1}_{tp_proxy_2}_{atype}_mon_esales_sd_{str(k)}_segment'] == e_seg) &
                (usag_df[f'{tp_proxy_1}_{tp_proxy_2}_{atype}_mon_nonesales_sd_{str(k)}_segment'] == n_seg)
            ]

            num_customers = subset.shape[0]
            esales_tp1 = subset[f'{tp1}_Total_esales'].sum()
            esales_tp2 = subset[f'{tp2}_Total_esales'].sum()
            nonesales_tp1 = subset[f'{tp1}_Total_nonesales'].sum()
            nonesales_tp2 = subset[f'{tp2}_Total_nonesales'].sum()

            nonesales_inc = subset[f'{tp1}_{tp2}_nonesales_incremental_abs'].sum()

            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, '.')] = num_customers
            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, f'non-eSales {tp1}')] = f"{nonesales_tp1:.0f}"
            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, f'non-eSales {tp2}')] = f"{nonesales_tp2:.0f}"
            cross_tab.loc[(e_seg, 'noneSales_Inc'), (n_seg, f'non-eSales {tp1}')] = f"{(nonesales_tp1/total_nonesales_tp1):.2%}"
            cross_tab.loc[(e_seg, 'noneSales_Inc'), (n_seg, f'non-eSales {tp2}')] = f"{(nonesales_tp2/total_nonesales_tp2):.2%}"

            cross_tab.loc[(e_seg, 'noneSales_Inc'), (n_seg, '.')] = f"{nonesales_inc:.0f}"

            cross_tab.loc[(e_seg, f'eSales {tp1}'), (n_seg, '.')] = f"{esales_tp1:.0f}"
            cross_tab.loc[(e_seg, f'eSales {tp2}'), (n_seg, '.')] = f"{esales_tp2:.0f}"
            cross_tab.loc[(e_seg, f'eSales {tp1}'), (n_seg, '..')] = f"{(esales_tp1/total_esales_tp1):.2%}"
            cross_tab.loc[(e_seg, f'eSales {tp2}'), (n_seg, '..')] = f"{(esales_tp2/total_esales_tp2):.2%}"


    return cross_tab

In [69]:
sheet_dict = {}
for atype in atypes:
    for k in ks:
        sheet_name = f"cross_tab_{tp1}_{tp2}_{atype}_mon_sd_{str(k)}"
        sheet_dict[sheet_name] = crosstab_func(usag_df,atype, k,tp1, tp2,tp_proxy_1,tp_proxy_2)
print("done")

done


In [70]:
sheet_dict[f'cross_tab_{tp1}_{tp2}_act_mon_sd_1']

Unallocated                             \
                                         .     .. non-eSales 24_jun_9   
Unallocated    # of Customers          891    NaN               48203   
               noneSales_Inc        105905    NaN               0.00%   
               eSales 24_jun_9       24794  0.00%                 NaN   
               eSales 25_jun_9     5031178  0.33%                 NaN   
New            # of Customers         1518    NaN              331404   
               noneSales_Inc        314812    NaN               0.01%   
               eSales 24_jun_9         274  0.00%                 NaN   
               eSales 25_jun_9     2206816  0.14%                 NaN   
Growing        # of Customers          484    NaN               45602   
               noneSales_Inc        176437    NaN               0.00%   
               eSales 24_jun_9      157432  0.01%                 NaN   
               eSales 25_jun_9     1790009  0.12%                 NaN   
Stable         # of Customers         1087    NaN               99035   
               noneSales_Inc         60625    NaN               0.00%   
               eSales 24_jun_9     1173490  0.08%                 NaN   
               eSales 25_jun_9     1640751  0.11%                 NaN   
Declining      # of Customers          741    NaN               67600   
               noneSales_Inc         21954    NaN               0.00%   
               eSales 24_jun_9      332510  0.02%                 NaN   
               eSales 25_jun_9      120705  0.01%                 NaN   
Not in Channel # of Customers        94607    NaN            11940043   
               noneSales_Inc       2937183    NaN               0.25%   
               eSales 24_jun_9         122  0.00%                 NaN   
               eSales 25_jun_9         727  0.00%                 NaN   

                                                         New         \
                               non-eSales 25_jun_9         .     ..   
Unallocated    # of Customers              2855520       537    NaN   
               noneSales_Inc                 0.06%         0    NaN   
               eSales 24_jun_9                 NaN     17872  0.00%   
               eSales 25_jun_9                 NaN   1195699  0.08%   
New            # of Customers              3133264     15306    NaN   
               noneSales_Inc                 0.07%     19344    NaN   
               eSales 24_jun_9                 NaN       458  0.00%   
               eSales 25_jun_9                 NaN  40038081  2.59%   
Growing        # of Customers               679778       324    NaN   
               noneSales_Inc                 0.01%         0    NaN   
               eSales 24_jun_9                 NaN    102283  0.01%   
               eSales 25_jun_9                 NaN    802917  0.05%   
Stable         # of Customers              1028721       628    NaN   
               noneSales_Inc                 0.02%        74    NaN   
               eSales 24_jun_9                 NaN    550425  0.04%   
               eSales 25_jun_9                 NaN    744050  0.05%   
Declining      # of Customers               289302       505    NaN   
               noneSales_Inc                 0.01%       185    NaN   
               eSales 24_jun_9                 NaN    185479  0.01%   
               eSales 25_jun_9                 NaN     49281  0.00%   
Not in Channel # of Customers             46332969    252644    NaN   
               noneSales_Inc                 0.99%      8991    NaN   
               eSales 24_jun_9                 NaN       413  0.00%   
               eSales 25_jun_9                 NaN       866  0.00%   

                                                                        \
                               non-eSales 24_jun_9 non-eSales 25_jun_9   
Unallocated    # of Customers                    0              304270   
               noneSales_Inc                 0.00%             

In [71]:
with pd.ExcelWriter(f"crosstab_output_bucket_pne_{tp1}_{tp2}.xlsx") as writer:
    for sheet_name, df in sheet_dict.items():
        df.to_excel(writer,sheet_name = sheet_name[:31])

### Placeholder